In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
import xgboost as xgb
import logging

In [2]:
import pickle
from tqdm import tqdm, tqdm_notebook, _tqdm_notebook, tqdm_pandas

In [3]:
log_fmt = "[%(asctime)s] %(levelname)s in %(module)s: %(message)s"
logging.basicConfig(format=log_fmt, level=logging.INFO)

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
data = pd.read_csv('../data2/data_1219.txt', sep='\t')
data.head()

,day,follow_topic,freq,gender,hour,inter_topic,label,q_ans_count,q_diff_qa_days_max,q_diff_qa_days_mean,...,uid_enc_count,qid_enc_count,gender_count,freq_count,uf_c1_count,uf_c2_count,uf_c3_count,uf_c4_count,uf_c5_count,wk
0,3865,-0.122170,4.0,3,22,0.021705,0.0,NaN,NaN,NaN,...,0.000000,0.000369,1.000000,0.960571,0.022114,0.371930,0.218308,0.062702,1.0,1
1,3862,-0.015598,4.0,3,15,-0.182562,0.0,NaN,NaN,NaN,...,0.000004,0.000036,1.000000,0.960571,0.024451,0.111552,0.444739,0.002961,1.0,5
2,3867,0.000000,3.0,3,4,0.070942,0.0,32.0,13.0,9.53125,...,0.000022,0.001441,1.000000,0.290809,0.006221,0.034154,0.402727,0.044313,1.0,3
3,3861,-0.302475,5.0,2,20,-0.191518,0.0,3.0,234.0,222.33333,...,0.000011,0.000018,0.000000,1.000000,0.024451,0.111552,0.444739,0.002961,1.0,4
4,3862,-0.095008,4.0,1,8,-0.176807,1.0,NaN,NaN,NaN,...,0.000043,0.000027,0.008695,0.960571,0.005739,0.007515,0.438648,0.125989,1.0,5


In [14]:
feature_cols = [x for x in data.columns if x not in ('label', 'uid', 'qid', 'dt', 'day')]

In [15]:
train_label=2593669

In [16]:
data.columns.values.tolist()

['day',
 'follow_topic',
 'freq',
 'gender',
 'hour',
 'inter_topic',
 'label',
 'q_ans_count',
 'q_diff_qa_days_max',
 'q_diff_qa_days_mean',
 'q_diff_qa_days_sum',
 'q_has_img_max',
 'q_has_img_mean',
 'q_has_img_sum',
 'q_has_video_max',
 'q_has_video_mean',
 'q_has_video_sum',
 'q_inv_count',
 'q_inv_mean',
 'q_inv_std',
 'q_inv_sum',
 'q_is_dest_max',
 'q_is_dest_mean',
 'q_is_dest_sum',
 'q_is_good_max',
 'q_is_good_mean',
 'q_is_good_sum',
 'q_is_rec_max',
 'q_is_rec_mean',
 'q_is_rec_sum',
 'q_reci_cheer_max',
 'q_reci_cheer_mean',
 'q_reci_cheer_sum',
 'q_reci_comment_max',
 'q_reci_comment_mean',
 'q_reci_comment_sum',
 'q_reci_dis_max',
 'q_reci_dis_mean',
 'q_reci_dis_sum',
 'q_reci_mark_max',
 'q_reci_mark_mean',
 'q_reci_mark_sum',
 'q_reci_no_help_max',
 'q_reci_no_help_mean',
 'q_reci_no_help_sum',
 'q_reci_tks_max',
 'q_reci_tks_mean',
 'q_reci_tks_sum',
 'q_reci_uncheer_max',
 'q_reci_uncheer_mean',
 'q_reci_uncheer_sum',
 'q_reci_xxx_max',
 'q_reci_xxx_mean',
 'q_rec

In [18]:
# target编码
logging.info("feature size %s", len(feature_cols))

X_train_all = data.iloc[:train_label][feature_cols]
y_train_all = data.iloc[:train_label]['label']
# test = data.iloc[len(train_label):]
# # del data
# assert len(test) == sub_size

# logging.info("train shape %s", train_label.shape)

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for index, (train_idx, val_idx) in enumerate(fold.split(X=X_train_all, y=y_train_all)):
    break

X_train, X_val, y_train, y_val = X_train_all.iloc[train_idx][feature_cols], X_train_all.iloc[val_idx][feature_cols], \
                                 y_train_all.iloc[train_idx], \
                                 y_train_all.iloc[val_idx]
del X_train_all

model_xgb = xgb.XGBClassifier(n_estimators=3000, n_jobs=-1,max_depth=5, objective='binary:logistic', seed=1000, silent=True)
model_xgb.fit(X_train, y_train,
              eval_metric=['logloss', 'auc'],
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=50)

[2019-12-19 19:09:51,347] INFO in <ipython-input-18-42a6bd206263>: feature size 128


[0]	validation_0-logloss:0.64294	validation_0-auc:0.735399
Multiple eval metrics have been passed: 'validation_0-auc' will be used for early stopping.

Will train until validation_0-auc hasn't improved in 50 rounds.
[1]	validation_0-logloss:0.601709	validation_0-auc:0.754291
[2]	validation_0-logloss:0.567522	validation_0-auc:0.760518
[3]	validation_0-logloss:0.538798	validation_0-auc:0.769213
[4]	validation_0-logloss:0.514544	validation_0-auc:0.770984
[5]	validation_0-logloss:0.493847	validation_0-auc:0.7735
[6]	validation_0-logloss:0.476332	validation_0-auc:0.774851
[7]	validation_0-logloss:0.461345	validation_0-auc:0.775607
[8]	validation_0-logloss:0.448459	validation_0-auc:0.778046
[9]	validation_0-logloss:0.437302	validation_0-auc:0.782623
[10]	validation_0-logloss:0.427544	validation_0-auc:0.783359
[11]	validation_0-logloss:0.419144	validation_0-auc:0.786643
[12]	validation_0-logloss:0.411946	validation_0-auc:0.788572
[13]	validation_0-logloss:0.405318	validation_0-auc:0.789206
[1

[132]	validation_0-logloss:0.338545	validation_0-auc:0.826225
[133]	validation_0-logloss:0.338505	validation_0-auc:0.826275
[134]	validation_0-logloss:0.33849	validation_0-auc:0.826283
[135]	validation_0-logloss:0.338464	validation_0-auc:0.826311
[136]	validation_0-logloss:0.338424	validation_0-auc:0.826361
[137]	validation_0-logloss:0.3384	validation_0-auc:0.82639
[138]	validation_0-logloss:0.338337	validation_0-auc:0.826454
[139]	validation_0-logloss:0.338259	validation_0-auc:0.826551
[140]	validation_0-logloss:0.338243	validation_0-auc:0.826571
[141]	validation_0-logloss:0.338217	validation_0-auc:0.826599
[142]	validation_0-logloss:0.338174	validation_0-auc:0.826661
[143]	validation_0-logloss:0.338162	validation_0-auc:0.826679
[144]	validation_0-logloss:0.338141	validation_0-auc:0.826708
[145]	validation_0-logloss:0.33811	validation_0-auc:0.82674
[146]	validation_0-logloss:0.338089	validation_0-auc:0.826767
[147]	validation_0-logloss:0.337911	validation_0-auc:0.826967
[148]	validati

[265]	validation_0-logloss:0.334472	validation_0-auc:0.831075
[266]	validation_0-logloss:0.334439	validation_0-auc:0.831119
[267]	validation_0-logloss:0.334402	validation_0-auc:0.83116
[268]	validation_0-logloss:0.334386	validation_0-auc:0.831177
[269]	validation_0-logloss:0.334369	validation_0-auc:0.831192
[270]	validation_0-logloss:0.33431	validation_0-auc:0.831267
[271]	validation_0-logloss:0.334293	validation_0-auc:0.831288
[272]	validation_0-logloss:0.334286	validation_0-auc:0.831297
[273]	validation_0-logloss:0.334285	validation_0-auc:0.831298
[274]	validation_0-logloss:0.334279	validation_0-auc:0.831305
[275]	validation_0-logloss:0.334273	validation_0-auc:0.831313
[276]	validation_0-logloss:0.334238	validation_0-auc:0.831358
[277]	validation_0-logloss:0.334223	validation_0-auc:0.83138
[278]	validation_0-logloss:0.334175	validation_0-auc:0.831439
[279]	validation_0-logloss:0.334151	validation_0-auc:0.831471
[280]	validation_0-logloss:0.334144	validation_0-auc:0.831483
[281]	valid

[398]	validation_0-logloss:0.332671	validation_0-auc:0.833213
[399]	validation_0-logloss:0.332664	validation_0-auc:0.833221
[400]	validation_0-logloss:0.332658	validation_0-auc:0.833231
[401]	validation_0-logloss:0.332658	validation_0-auc:0.83323
[402]	validation_0-logloss:0.332646	validation_0-auc:0.833242
[403]	validation_0-logloss:0.332627	validation_0-auc:0.83326
[404]	validation_0-logloss:0.332619	validation_0-auc:0.83327
[405]	validation_0-logloss:0.332612	validation_0-auc:0.833279
[406]	validation_0-logloss:0.332604	validation_0-auc:0.833287
[407]	validation_0-logloss:0.332588	validation_0-auc:0.833307
[408]	validation_0-logloss:0.332568	validation_0-auc:0.833328
[409]	validation_0-logloss:0.332561	validation_0-auc:0.833336
[410]	validation_0-logloss:0.332551	validation_0-auc:0.833348
[411]	validation_0-logloss:0.332527	validation_0-auc:0.83338
[412]	validation_0-logloss:0.332522	validation_0-auc:0.833386
[413]	validation_0-logloss:0.33252	validation_0-auc:0.83339
[414]	validati

...

[2503]	validation_0-logloss:0.326856	validation_0-auc:0.840049
[2504]	validation_0-logloss:0.326856	validation_0-auc:0.84005
[2505]	validation_0-logloss:0.32685	validation_0-auc:0.840056
[2506]	validation_0-logloss:0.32685	validation_0-auc:0.840057
[2507]	validation_0-logloss:0.326849	validation_0-auc:0.840058
[2508]	validation_0-logloss:0.326841	validation_0-auc:0.840069
[2509]	validation_0-logloss:0.326842	validation_0-auc:0.840066
[2510]	validation_0-logloss:0.326844	validation_0-auc:0.840064
[2511]	validation_0-logloss:0.326843	validation_0-auc:0.840065
[2512]	validation_0-logloss:0.32684	validation_0-auc:0.840068
[2513]	validation_0-logloss:0.326842	validation_0-auc:0.840066
[2514]	validation_0-logloss:0.326839	validation_0-auc:0.840068
[2515]	validation_0-logloss:0.326837	validation_0-auc:0.84007
[2516]	validation_0-logloss:0.326836	validation_0-auc:0.840072
[2517]	validation_0-logloss:0.326836	validation_0-auc:0.840071
[2518]	validation_0-logloss:0.32683	validation_0-auc:0.84007

[2634]	validation_0-logloss:0.326709	validation_0-auc:0.840225
[2635]	validation_0-logloss:0.32671	validation_0-auc:0.840224
[2636]	validation_0-logloss:0.32671	validation_0-auc:0.840223
[2637]	validation_0-logloss:0.326711	validation_0-auc:0.840223
[2638]	validation_0-logloss:0.326711	validation_0-auc:0.840223
[2639]	validation_0-logloss:0.326712	validation_0-auc:0.840221
[2640]	validation_0-logloss:0.326712	validation_0-auc:0.840222
[2641]	validation_0-logloss:0.326712	validation_0-auc:0.840222
[2642]	validation_0-logloss:0.326712	validation_0-auc:0.840222
[2643]	validation_0-logloss:0.326711	validation_0-auc:0.840224
[2644]	validation_0-logloss:0.326711	validation_0-auc:0.840224
[2645]	validation_0-logloss:0.326711	validation_0-auc:0.840224
[2646]	validation_0-logloss:0.326712	validation_0-auc:0.840221
[2647]	validation_0-logloss:0.32671	validation_0-auc:0.840223
[2648]	validation_0-logloss:0.326712	validation_0-auc:0.840221
[2649]	validation_0-logloss:0.326712	validation_0-auc:0.84

[2765]	validation_0-logloss:0.326665	validation_0-auc:0.840279
[2766]	validation_0-logloss:0.326666	validation_0-auc:0.840279
[2767]	validation_0-logloss:0.326666	validation_0-auc:0.840282
[2768]	validation_0-logloss:0.326666	validation_0-auc:0.840281
[2769]	validation_0-logloss:0.326664	validation_0-auc:0.840283
[2770]	validation_0-logloss:0.326665	validation_0-auc:0.840282
[2771]	validation_0-logloss:0.326668	validation_0-auc:0.840277
[2772]	validation_0-logloss:0.326668	validation_0-auc:0.840279
[2773]	validation_0-logloss:0.326665	validation_0-auc:0.840283
[2774]	validation_0-logloss:0.326664	validation_0-auc:0.840283
[2775]	validation_0-logloss:0.326663	validation_0-auc:0.840285
[2776]	validation_0-logloss:0.326662	validation_0-auc:0.840286
[2777]	validation_0-logloss:0.326662	validation_0-auc:0.840287
[2778]	validation_0-logloss:0.326659	validation_0-auc:0.840291
[2779]	validation_0-logloss:0.326658	validation_0-auc:0.840293
[2780]	validation_0-logloss:0.326659	validation_0-auc:0

[2896]	validation_0-logloss:0.326562	validation_0-auc:0.840419
[2897]	validation_0-logloss:0.32656	validation_0-auc:0.840423
[2898]	validation_0-logloss:0.326559	validation_0-auc:0.840423
[2899]	validation_0-logloss:0.326558	validation_0-auc:0.840425
[2900]	validation_0-logloss:0.326557	validation_0-auc:0.840427
[2901]	validation_0-logloss:0.326557	validation_0-auc:0.840426
[2902]	validation_0-logloss:0.326558	validation_0-auc:0.840425
[2903]	validation_0-logloss:0.326559	validation_0-auc:0.840422
[2904]	validation_0-logloss:0.32656	validation_0-auc:0.840421
[2905]	validation_0-logloss:0.326558	validation_0-auc:0.840424
[2906]	validation_0-logloss:0.326556	validation_0-auc:0.840424
[2907]	validation_0-logloss:0.326555	validation_0-auc:0.840427
[2908]	validation_0-logloss:0.326556	validation_0-auc:0.840426
[2909]	validation_0-logloss:0.326555	validation_0-auc:0.840427
[2910]	validation_0-logloss:0.326555	validation_0-auc:0.840427
[2911]	validation_0-logloss:0.326554	validation_0-auc:0.8

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=3000, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1000,
              silent=True, subsample=1, verbosity=1)